In [1]:
import pandas as pd
import glob
import numpy as np
from pathlib import Path

# 1. Read manual labelling data

In [2]:
Path().cwd()

PosixPath('/home/kirilltobola/Dev/github.com/ru-facts-200/cta')

In [3]:
labeling = pd.read_csv("labeled_tables.csv", sep=";", engine="c")
labeling

,table_id,column_id,label_id,label
0,1_aviation_table,0,123.0,самолет
1,1_aviation_table,1,108.0,производитель
2,1_aviation_table,2,101.0,примечание
3,2_aviation_table,0,123.0,самолет
4,2_aviation_table,1,42.0,изображение
...,...,...,...,...
972,7_energy_table,6,52.0,количество
973,7_energy_table,7,26.0,дата
974,7_energy_table,8,26.0,дата
975,7_energy_table,9,26.0,дата


In [4]:
# remove columns with na values
labeling = labeling[~labeling["label"].isna()]
labeling

,table_id,column_id,label_id,label
0,1_aviation_table,0,123.0,самолет
1,1_aviation_table,1,108.0,производитель
2,1_aviation_table,2,101.0,примечание
3,2_aviation_table,0,123.0,самолет
4,2_aviation_table,1,42.0,изображение
...,...,...,...,...
972,7_energy_table,6,52.0,количество
973,7_energy_table,7,26.0,дата
974,7_energy_table,8,26.0,дата
975,7_energy_table,9,26.0,дата


In [5]:
# cast label_id to integer type
labeling["label_id"] = labeling["label_id"].astype(int)
labeling

/tmp/ipykernel_3795/2307969231.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labeling["label_id"] = labeling["label_id"].astype(int)


,table_id,column_id,label_id,label
0,1_aviation_table,0,123,самолет
1,1_aviation_table,1,108,производитель
2,1_aviation_table,2,101,примечание
3,2_aviation_table,0,123,самолет
4,2_aviation_table,1,42,изображение
...,...,...,...,...
972,7_energy_table,6,52,количество
973,7_energy_table,7,26,дата
974,7_energy_table,8,26,дата
975,7_energy_table,9,26,дата


In [6]:
# number of unique tables:
labeling["table_id"].unique().shape

(200,)

# 2. Read column data from RF-200

In [7]:
current_directory = Path.cwd()
current_directory.parent / 'csv'

PosixPath('/home/kirilltobola/Dev/github.com/ru-facts-200/csv')

In [8]:
current_directory = Path.cwd()
current_directory.parent
data_path = current_directory.parent / 'csv' # Path("csv/")


def concat_to_str(input: list) -> str:
    filtered_input = []
    for i in input:
        i = str(i)
        if i == None or i == np.nan or i == "nan" or i == "" or i.isspace():
            continue
        filtered_input.append(str(i))
    return " ".join(filtered_input)


traverse = []
for table_id in labeling["table_id"].unique():
    paths = glob.glob(f"**/*{table_id}.csv", root_dir=data_path, recursive=True)
    for path in paths:
        if "all-tables" not in path:
            table_path = path
    assert table_path is not None

    # print(table_path)
    table = pd.read_csv(data_path / table_path, sep="|")
    for column_id, column in enumerate(table.columns):
        traverse.append(
            (table_id, column_id, table[column].tolist())
        )
df = pd.DataFrame(traverse, columns=['table_id', 'column_id', 'column_data'])
df["column_data"] = df["column_data"].apply(lambda x: concat_to_str(x)); df


,table_id,column_id,column_data
0,1_aviation_table,0,Z-5 Z-6 Z-7 Z-8 Z-9 Z-19 WZ-10 Z-11 EC-120 Z-1...
1,1_aviation_table,1,Harbin Changhe Changhe Harbin Harbin Changhe C...
2,1_aviation_table,2,"Копия Ми-4 , снят с вооружения Экспортные Ми-8..."
3,2_aviation_table,0,Истребители Обозначение P-40 P-39 Airacobra P-...
4,2_aviation_table,1,Изображение 150px 150px 150px 150px 150px 150p...
...,...,...,...
970,7_energy_table,6,Таблица 1. Парк реакторов АЭС Японии с 01.12.1...
971,7_energy_table,7,Таблица 1. Парк реакторов АЭС Японии с 01.12.1...
972,7_energy_table,8,Таблица 1. Парк реакторов АЭС Японии с 01.12.1...
973,7_energy_table,9,Таблица 1. Парк реакторов АЭС Японии с 01.12.1...


# 3. Merge manual labelling with column data

In [9]:
merged_df = labeling.merge(df, on=["table_id", "column_id"])
merged_df
# Some columns were removed (because label is not set for these columns)

,table_id,column_id,label_id,label,column_data
0,1_aviation_table,0,123,самолет,Z-5 Z-6 Z-7 Z-8 Z-9 Z-19 WZ-10 Z-11 EC-120 Z-1...
1,1_aviation_table,1,108,производитель,Harbin Changhe Changhe Harbin Harbin Changhe C...
2,1_aviation_table,2,101,примечание,"Копия Ми-4 , снят с вооружения Экспортные Ми-8..."
3,2_aviation_table,0,123,самолет,Истребители Обозначение P-40 P-39 Airacobra P-...
4,2_aviation_table,1,42,изображение,Изображение 150px 150px 150px 150px 150px 150p...
...,...,...,...,...,...
922,7_energy_table,6,52,количество,Таблица 1. Парк реакторов АЭС Японии с 01.12.1...
923,7_energy_table,7,26,дата,Таблица 1. Парк реакторов АЭС Японии с 01.12.1...
924,7_energy_table,8,26,дата,Таблица 1. Парк реакторов АЭС Японии с 01.12.1...
925,7_energy_table,9,26,дата,Таблица 1. Парк реакторов АЭС Японии с 01.12.1...


# 4. Save merged dataframe

In [10]:
merged_df.to_csv("rf200-cta-test.csv", sep="|", index=False)